In [5]:
import numpy as np
import pandas as pd

In [6]:
data = pd.read_csv(fr"/Users/jordan/Documents/BarclaysStage/portfolio_analysis/positions.csv",index_col=0)

In [7]:
data

,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local
stock_id,,,,,,,,,,,,,
1,equity_1,AGI,DEU,EUR,Information Technology,Software & Services,Systems Software,1.735624,10043,SHORT,-4810,145.19,147.06
2,equity_2,PWH,RUS,USD,Consumer Staples,Food Beverage & Tobacco,Meat Poultry & Fish,0.675354,640830,LONG,38342,112.48,100.28
3,equity_3,NDE,DEU,USD,Industrials,Capital Goods,Heavy Electrical Equipment,0.699394,36094,SHORT,-19881,43.36,41.56
4,equity_4,ABB,USA,USD,Financials,Insurance,Reinsurance,2.215079,135786553,SHORT,-100194,24.79,26.38
5,equity_5,WRF,RUS,USD,Utilities,Utilities,Electric Utilities,1.259133,26094,LONG,3963,149.90,149.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,equity_2996,HSH,CAN,CAD,Information Technology,Technology Hardware & Equipment,Electronic Manufacturing Services,1.723244,3193,SHORT,-232,238.13,249.60
2997,equity_2997,QLG,BEL,EUR,Utilities,Utilities,Gas Utilities,0.977303,291651784,LONG,77866,11.07,10.13
2998,equity_2998,YOV,DEU,USD,Utilities,Utilities,Gas Utilities,1.186587,1955,LONG,3743,141.71,160.19


In [7]:
import pandas as pd, numpy as np
from pathlib import Path
from datetime import datetime

# ---------- Config ----------
BASE_CCY = "USD"
FX = {  # fill with real rates if given; else keep 1.0 and label in the report
    # "EUR": 1.08, "GBP": 1.30, "JPY": 0.0063, ...
}
THR = dict(beta_drift=0.015, ccy=0.03, bucket=0.05, single_name=0.03, pctadv=0.10)

# ---------- Load ----------
src = Path("positions.csv")              # your file path
out_txt = Path("report.txt")
out_xlsx = Path("report_tables.xlsx")

df = pd.read_csv(src)
df.columns = (df.columns.str.strip().str.lower().str.replace(r"[\s\-]+","_", regex=True))

# ---------- Derived fields ----------
df["shares"] = pd.to_numeric(df["posn_shares"], errors="coerce")
df["price"]  = pd.to_numeric(df["market_price_local"], errors="coerce")
df["cost"]   = pd.to_numeric(df["cost_basis_local"], errors="coerce")
df["beta"]   = pd.to_numeric(df["beta"], errors="coerce")
df["adv"]    = pd.to_numeric(df["avg_daily_volume"], errors="coerce")

# FX to base (fallback = 1.0)
df["fx_to_base"] = df["currency"].map(FX).fillna(1.0)

# Notional & MV (base)
df["mv_base"] = df["shares"] * df["price"] * df["fx_to_base"]
df["beta_dollars"] = df["beta"] * df["mv_base"]

# Liquidity & PnL
df["pct_adv"] = np.where(df["adv"]>0, np.abs(df["shares"])/df["adv"], np.nan)
df["unrl_pnl"] = (df["price"] - df["cost"]) * df["shares"]
df["unrl_pnl_pct"] = np.where(df["cost"]!=0, df["price"]/df["cost"] - 1.0, np.nan)

# ---------- KPIs ----------
gross = df["mv_base"].abs().sum()
net   = df["mv_base"].sum()
long_gross  = df.loc[df["mv_base"]>0, "mv_base"].sum()
short_gross = df.loc[df["mv_base"]<0, "mv_base"].abs().sum()
long_pct  = long_gross/(gross or 1.0)
short_pct = short_gross/(gross or 1.0)
net_beta  = df["beta_dollars"].sum()
beta_drift = abs(net_beta)/(gross or 1.0)

def gsum(by):
    g = df.groupby(by, dropna=False).agg(
        net=("mv_base","sum"),
        gross=("mv_base", lambda s: s.abs().sum()),
        beta_dollars=("beta_dollars","sum"),
        names=("ticker","nunique")
    ).reset_index()
    g["pct_gross"] = g["gross"]/(gross or 1.0)
    return g.sort_values("pct_gross", ascending=False)

g_ccy  = gsum("currency")
g_ctry = gsum("country")
g_sect = gsum("sector")

# ---------- Flags ----------
flags = []
if beta_drift > THR["beta_drift"]:
    flags.append(f"Market beta drift {beta_drift:.2%} > {THR['beta_drift']:.2%}")

for label, g, thr in [("Currency", g_ccy, THR["ccy"]),
                      ("Country",  g_ctry, THR["bucket"]),
                      ("Sector",   g_sect, THR["bucket"])]:
    skew = g.loc[g["net"].abs() > thr * (gross or 1.0)]
    for _, r in skew.iterrows():
        flags.append(f"{label} skew {r[label.lower()]}: net {r['net']:.0f} ({r['net']/(gross or 1.0):.2%} of gross)")

big = df.assign(w=lambda x: x["mv_base"].abs()/(gross or 1.0)).query("w > @THR['single_name']")
for _, r in big.iterrows():
    flags.append(f"Single-name concentration {r['ticker']}: {r['w']:.2%} of gross")

liq = df.query("pct_adv > @THR['pctadv']")
for _, r in liq.iterrows():
    flags.append(f"Liquidity: {r['ticker']} at {r['pct_adv']:.1%} of ADV")

# ---------- Write plaintext ----------
def money(x): 
    try: return f"{x:,.0f}"
    except: return str(x)

lines = []
fx_note = "real FX applied" if any(v!=1.0 for v in df["fx_to_base"].unique()) else "FX=1.0 placeholder (exposures approximate)"
lines.append(f"FACTOR-NEUTRAL EQUITIES REPORT | {datetime.now():%Y-%m-%d %H:%M} | Base: {BASE_CCY} ({fx_note})")
lines.append(f"Rows: {len(df)}")
lines.append("")
lines.append("Portfolio KPIs")
lines.append(f"  Gross: {money(gross)} | Net: {money(net)} | Long%: {long_pct:.1%} | Short%: {short_pct:.1%}")
lines.append(f"  Net beta-$: {money(net_beta)} | Beta drift: {beta_drift:.2%} of gross")
lines.append(f"  Liquidity (>%ADV 10%): {(df.loc[df['pct_adv']>0.10,'mv_base'].abs().sum()/(gross or 1.0)):.1%} | "
             f"Top-10 names: {df['mv_base'].abs().nlargest(10).sum()/(gross or 1.0):.1%}")
lines.append("")
for title, g in [("Top Currency Buckets", g_ccy.head(10)),
                 ("Top Country Buckets",  g_ctry.head(10)),
                 ("Top Sector Buckets",   g_sect.head(10))]:
    t = g.copy()
    t["net"] = t["net"].map(money); t["gross"] = t["gross"].map(money)
    t["beta_dollars"] = t["beta_dollars"].map(money)
    t["pct_gross"] = (t["pct_gross"]*100).map(lambda v: f"{v:.1f}%")
    lines.append(title)
    lines.append(t.to_string(index=False))
    lines.append("")

lines.append("Watchlist Flags")
lines += [f"  - {f}" for f in flags] or ["  (None)"]
out_txt.write_text("\n".join(lines))

# ---------- Workbook ----------
with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as xw:
    g_ccy.to_excel(xw, "by_currency", index=False)
    g_ctry.to_excel(xw, "by_country", index=False)
    g_sect.to_excel(xw, "by_sector", index=False)
    (df.assign(abs_beta=lambda x: x["beta_dollars"].abs())
       .sort_values("abs_beta", ascending=False)
       .head(20)
       .drop(columns="abs_beta")
       ).to_excel(xw, "top_beta_contrib", index=False)
    (df.sort_values("mv_base", key=lambda s: s.abs(), ascending=False)
       .head(20)
       ).to_excel(xw, "top_exposures", index=False)
    df.sort_values("pct_adv", ascending=False).to_excel(xw, "liquidity", index=False)
    df.to_excel(xw, "positions_enriched", index=False)

print("Wrote:", out_txt, "and", out_xlsx)


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Factor-neutral global equities daily report
-------------------------------------------
Reads a positions CSV and produces:
  - report.txt (plaintext pre-open summary)
  - report_tables.xlsx (drill-down tabs)

Assumptions:
- CSV columns (case/space-insensitive, flexible names):
  stock_id, name, ticker, country, currency, sector, beta,
  avg_daily_volume, side, posn_shares, cost_basis_local, market_price_local
- ADV is in shares/day. If it's in notional, switch %ADV formula accordingly.
- FX to base is provided via --fx optional CSV (columns: currency,to_base).
  If omitted, FX=1.0 and we clearly label that in the report.

Usage:
  python build_report.py --positions positions.csv --base USD --fx fx_latest.csv
"""

from __future__ import annotations

import argparse
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd


# ----------------------------- Configuration ----------------------------- #

@dataclass
class Thresholds:
    beta_drift: float = 0.015       # 1.5% of gross
    currency_skew: float = 0.03     # 3% of gross
    bucket_skew: float = 0.05       # 5% (country/sector)
    single_name: float = 0.03       # 3% of gross
    pct_adv_flag: float = 0.10      # 10% of ADV
    liq_adjust_tau: float = 0.10    # 10% for liquidity-adjusted gross


# ----------------------------- Utilities -------------------------------- #

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column names to snake_case lowercase for robust mapping."""
    out = df.copy()
    out.columns = (
        out.columns
        .str.strip()
        .str.lower()
        .str.replace(r"[\s\-]+", "_", regex=True)
    )
    return out


def pick(df: pd.DataFrame, choices: List[str], required: bool = True) -> Optional[str]:
    """Pick the first column present from a list of candidate names."""
    cols = set(df.columns)
    for c in choices:
        if c in cols:
            return c
    if required:
        raise KeyError(f"Missing required column; tried: {choices}")
    return None


def load_fx_map(path: Optional[Path]) -> Dict[str, float]:
    """
    Load FX rates if provided (CSV with columns ['currency','to_base']).
    Return mapping {currency: to_base}. If path is None, return empty mapping.
    """
    if path is None:
        return {}
    fx = pd.read_csv(path)
    fx = normalize_columns(fx)
    if not {"currency", "to_base"}.issubset(fx.columns):
        raise ValueError("FX file must have columns: currency,to_base")
    return dict(zip(fx["currency"], fx["to_base"]))


# ----------------------------- Core pipeline ----------------------------- #

def load_positions(path: Path) -> pd.DataFrame:
    """Load and minimally validate positions CSV; map canonical column names."""
    df = pd.read_csv(path)
    df = normalize_columns(df)

    # Map flexible input names to canonical ones
    mapping = {
        "ticker": pick(df, ["ticker", "symbol", "ric", "bbg_ticker", "bbg", "sedol", "isin"], required=False),
        "name": pick(df, ["name", "security_name"], required=False),
        "stock_id": pick(df, ["stock_id", "id"], required=False),
        "country": pick(df, ["country", "country_code", "iso_country"], required=False),
        "currency": pick(df, ["currency", "ccy", "curr"], required=True),
        "sector": pick(df, ["sector", "gics_sector", "industry_group"], required=False),
        "industry": pick(df, ["industry"], required=False),
        "sub_industry": pick(df, ["sub_industry", "subindustry"], required=False),
        "beta": pick(df, ["beta", "capm_beta", "beta_to_index"], required=True),
        "adv": pick(df, ["avg_daily_volume", "average_daily_volume", "adv", "adv_shares"], required=True),
        "side": pick(df, ["side", "direction"], required=False),
        "shares": pick(df, ["posn_shares", "position_shares", "shares"], required=True),
        "cost": pick(df, ["cost_basis_local", "cost_basis", "avg_cost"], required=False),
        "price": pick(df, ["market_price_local", "last_price", "px_last", "close"], required=True),
    }

    # Rename present columns
    ren = {v: k for k, v in mapping.items() if v is not None}
    df = df.rename(columns=ren)

    # Coerce numerics
    for num in ["beta", "adv", "shares", "cost", "price"]:
        if num in df.columns:
            df[num] = pd.to_numeric(df[num], errors="coerce")

    # Fill missing optional classifiers for grouping
    for opt in ["ticker", "name", "stock_id", "country", "sector", "industry", "sub_industry", "side"]:
        if opt not in df.columns:
            df[opt] = "Unknown"

    return df


def enrich_positions(df: pd.DataFrame, fx_map: Dict[str, float]) -> pd.DataFrame:
    """Compute all derived fields used by analytics."""
    d = df.copy()

    # FX to base; default to 1.0 if missing currency in map
    d["fx_to_base"] = d["currency"].map(fx_map).fillna(1.0)

    # Local notional and Mark-to-Base (MV_base)
    d["notional_local"] = d["shares"] * d["price"]
    d["mv_base"] = d["notional_local"] * d["fx_to_base"]

    # Beta dollars (money-weighted beta exposure)
    d["beta_dollars"] = d["beta"] * d["mv_base"]

    # Liquidity: %ADV (shares)
    d["pct_adv"] = np.where(d["adv"] > 0, np.abs(d["shares"]) / d["adv"], np.nan)

    # Unrealized P&L (absolute and %); cost may be absent or partially missing
    if "cost" in d.columns and d["cost"].notna().any():
        d["unrl_pnl"] = (d["price"] - d["cost"]) * d["shares"]
        d["unrl_pnl_pct"] = np.where(d["cost"] != 0, d["price"] / d["cost"] - 1.0, np.nan)
    else:
        d["unrl_pnl"] = np.nan
        d["unrl_pnl_pct"] = np.nan

    # Helpful: absolute MV and gross weight placeholder (fill after totals)
    d["abs_mv_base"] = d["mv_base"].abs()

    return d


def portfolio_kpis(d: pd.DataFrame) -> Dict[str, float]:
    """Compute portfolio-level key performance indicators."""
    gross = float(d["abs_mv_base"].sum())
    net = float(d["mv_base"].sum())
    long_gross = float(d.loc[d["mv_base"] > 0, "mv_base"].sum())
    short_gross = float(d.loc[d["mv_base"] < 0, "mv_base"].abs().sum())
    long_pct = long_gross / gross if gross else 0.0
    short_pct = short_gross / gross if gross else 0.0
    net_beta = float(d["beta_dollars"].sum())
    beta_drift = abs(net_beta) / gross if gross else 0.0

    # Liquidity share of gross in %ADV > 10%
    liq_gross_share = float(d.loc[d["pct_adv"] > 0.10, "abs_mv_base"].sum()) / (gross or 1.0)

    # Top-10 concentration
    top10_weight = float(d["abs_mv_base"].nlargest(10).sum()) / (gross or 1.0)

    return dict(
        gross=gross, net=net,
        long_pct=long_pct, short_pct=short_pct,
        net_beta=net_beta, beta_drift=beta_drift,
        liq_gross_10pct=liq_gross_share,
        top10_weight=top10_weight
    )


def group_summary(d: pd.DataFrame, by: str) -> pd.DataFrame:
    """Aggregate Net, Gross, %Gross, Beta$, #names by a grouping dimension."""
    total_gross = d["abs_mv_base"].sum() or 1.0
    g = (
        d.groupby(by, dropna=False)
        .agg(
            net=("mv_base", "sum"),
            gross=("abs_mv_base", "sum"),
            beta_dollars=("beta_dollars", "sum"),
            names=("ticker", "nunique"),
        )
        .reset_index()
    )
    g["pct_gross"] = g["gross"] / total_gross
    g = g.sort_values("pct_gross", ascending=False)
    return g


def beta_buckets(d: pd.DataFrame, edges: List[float] = [-9, -0.5, 0, 0.5, 9]) -> pd.DataFrame:
    """Distribution of gross exposure across beta buckets."""
    # Bin betas; edges wide at ends to catch extremes
    labels = [f"[{edges[i]},{edges[i+1]})" for i in range(len(edges)-1)]
    b = d.copy()
    b["beta_bin"] = pd.cut(b["beta"], bins=edges, labels=labels, include_lowest=True)
    g = b.groupby("beta_bin", dropna=False).agg(
        gross=("abs_mv_base", "sum"),
        names=("ticker", "nunique")
    ).reset_index()
    total_gross = d["abs_mv_base"].sum() or 1.0
    g["pct_gross"] = g["gross"] / total_gross
    return g.sort_values("pct_gross", ascending=False)


def concentration_metrics(d: pd.DataFrame) -> Dict[str, float]:
    """Compute HHI concentration on absolute-MV weights."""
    gross = d["abs_mv_base"].sum() or 1.0
    w = d["abs_mv_base"] / gross
    hhi = float((w ** 2).sum())          # Herfindahl-Hirschman Index
    return dict(hhi=hhi)


def liquidity_adjusted_gross(d: pd.DataFrame, tau: float = 0.10) -> float:
    """
    Liquidity-adjusted gross: down-weight gross by %ADV/tau (cap at 1).
    Interprets tau=10% as 'full weight if %ADV >=10%, else proportional'.
    """
    scale = np.minimum(1.0, (d["pct_adv"] / tau).fillna(0.0))
    lag = float((d["abs_mv_base"] * scale).sum())
    return lag


def flags_and_watchlist(
    d: pd.DataFrame,
    kpis: Dict[str, float],
    g_ccy: pd.DataFrame,
    g_ctry: pd.DataFrame,
    g_sect: pd.DataFrame,
    thr: Thresholds,
) -> List[str]:
    """Generate textual flags based on thresholds."""
    out = []
    gross = kpis["gross"] or 1.0

    # Beta drift
    if kpis["beta_drift"] > thr.beta_drift:
        out.append(f"Market beta drift {kpis['beta_drift']:.2%} exceeds {thr.beta_drift:.2%} of gross")

    # Currency/Country/Sector skews
    for label, g, lim in [("Currency", g_ccy, thr.currency_skew),
                          ("Country", g_ctry, thr.bucket_skew),
                          ("Sector", g_sect, thr.bucket_skew)]:
        sk = g.loc[g["net"].abs() > lim * gross]
        for _, r in sk.iterrows():
            out.append(f"{label} skew: {r[ label.lower() ]} net={r['net']:.0f} ({r['net']/gross:.2%} of gross)")

    # Single-name concentration
    name_weights = d.assign(w=d["abs_mv_base"] / gross).query("w > @thr.single_name")
    for _, r in name_weights.iterrows():
        out.append(f"Single-name concentration: {r.get('ticker','<NA>')} at {r['w']:.2%} of gross")

    # Liquidity
    for _, r in d.loc[d["pct_adv"] > thr.pct_adv_flag].iterrows():
        out.append(f"Liquidity: {r.get('ticker','<NA>')} at {r['pct_adv']:.1%} of ADV")

    return out


def qa_checks(d: pd.DataFrame) -> List[str]:
    """Data quality checks; return list of warnings (empty if all good)."""
    warnings = []

    # Missing values in critical fields
    for col in ["beta", "adv", "price", "shares", "currency"]:
        if d[col].isna().any():
            n = int(d[col].isna().sum())
            warnings.append(f"{n} rows have missing '{col}'")

    # Non-positive ADV
    bad_adv = (d["adv"] <= 0) | d["adv"].isna()
    if bad_adv.any():
        warnings.append(f"{int(bad_adv.sum())} rows have non-positive or missing ADV (pct_adv set to NaN)")

    # Extreme betas
    extreme_beta = (d["beta"].abs() > 3).sum()
    if extreme_beta:
        warnings.append(f"{int(extreme_beta)} rows have |beta| > 3 (clip or review)")

    # Reconciliation of MV sums (here identity by construction; keep as note)
    # Additional checks can verify cost basis coherence, etc.

    return warnings


# ----------------------------- Output writers ---------------------------- #

def fmt_money(x: float) -> str:
    try:
        return f"{x:,.0f}"
    except Exception:
        return str(x)


def write_text_report(
    out_path: Path,
    base_ccy: str,
    used_real_fx: bool,
    d: pd.DataFrame,
    kpis: Dict[str, float],
    g_ccy: pd.DataFrame,
    g_ctry: pd.DataFrame,
    g_sect: pd.DataFrame,
    beta_bins: pd.DataFrame,
    lag: float,
    hhi: float,
    flags: List[str],
    qa: List[str],
) -> None:
    """Compose and write the plaintext pre-open report."""
    lines: List[str] = []
    fx_note = "real FX applied" if used_real_fx else "FX=1.0 placeholder (approximate cross-currency totals)"

    lines.append(f"FACTOR-NEUTRAL EQUITIES REPORT | Base: {base_ccy} | {fx_note}")
    lines.append(f"Rows: {len(d)}")
    lines.append("")

    # KPIs
    lines.append("Portfolio KPIs")
    lines.append(f"  Gross: {fmt_money(kpis['gross'])} | Net: {fmt_money(kpis['net'])} | "
                 f"Long%: {kpis['long_pct']:.1%} | Short%: {kpis['short_pct']:.1%}")
    lines.append(f"  Net beta-$: {fmt_money(kpis['net_beta'])} | Beta drift: {kpis['beta_drift']:.2%} of gross")
    lines.append(f"  Liquidity (>%ADV 10%): {kpis['liq_gross_10pct']:.1%} of gross | "
                 f"Top-10 names: {kpis['top10_weight']:.1%} of gross")
    lines.append(f"  Liquidity-adjusted gross (LAG): {fmt_money(lag)} | HHI: {hhi:.4f}")
    lines.append("")

    # Buckets (top)
    def block(title: str, g: pd.DataFrame, show: int = 8) -> List[str]:
        t = g.copy().head(show)
        t["net"] = t["net"].map(fmt_money)
        t["gross"] = t["gross"].map(fmt_money)
        t["beta_dollars"] = t["beta_dollars"].map(fmt_money)
        t["pct_gross"] = (t["pct_gross"] * 100).map(lambda v: f"{v:.1f}%")
        return [title, t.to_string(index=False), ""]

    lines += block("Top Currency Buckets (by % gross)", g_ccy)
    lines += block("Top Country Buckets (by % gross)", g_ctry)
    lines += block("Top Sector Buckets (by % gross)", g_sect)

    # Beta buckets
    t = beta_bins.copy()
    t["gross"] = t["gross"].map(fmt_money)
    t["pct_gross"] = (t["pct_gross"] * 100).map(lambda v: f"{v:.1f}%")
    lines.append("Beta Buckets (gross distribution)")
    lines.append(t.to_string(index=False))
    lines.append("")

    # Watchlist flags
    lines.append("Watchlist Flags")
    if flags:
        lines += [f"  - {f}" for f in flags]
    else:
        lines.append("  (None)")
    lines.append("")

    # QA warnings
    if qa:
        lines.append("Data Quality Notes")
        lines += [f"  - {w}" for w in qa]
        lines.append("")

    out_path.write_text("\n".join(lines), encoding="utf-8")


def write_excel_report(
    out_xlsx: Path,
    d: pd.DataFrame,
    g_ccy: pd.DataFrame,
    g_ctry: pd.DataFrame,
    g_sect: pd.DataFrame,
    beta_bins: pd.DataFrame,
) -> None:
    """Write drill-down workbook with pivots and enriched positions."""
    with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as xw:
        g_ccy.to_excel(xw, sheet_name="by_currency", index=False)
        g_ctry.to_excel(xw, sheet_name="by_country", index=False)
        g_sect.to_excel(xw, sheet_name="by_sector", index=False)

        # Top beta contributors (absolute)
        (d.assign(abs_beta=lambda x: x["beta_dollars"].abs())
           .sort_values("abs_beta", ascending=False)
           .drop(columns=["abs_beta"])
           .head(50)
           ).to_excel(xw, sheet_name="top_beta_contrib", index=False)

        # Top exposures by absolute MV
        (d.sort_values("abs_mv_base", ascending=False)
           .head(50)
           ).to_excel(xw, sheet_name="top_exposures", index=False)

        # Liquidity detail sorted by %ADV
        (d.sort_values("pct_adv", ascending=False)
           ).to_excel(xw, sheet_name="liquidity", index=False)

        # Beta bucket distribution
        beta_bins.to_excel(xw, sheet_name="beta_buckets", index=False)

        # Raw enriched positions
        d.to_excel(xw, sheet_name="positions_enriched", index=False)


# ----------------------------- Main CLI --------------------------------- #

def main():
    ap = argparse.ArgumentParser(description="Build factor-neutral equities daily report.")
    ap.add_argument("--positions", type=Path, required=True, help="Path to positions CSV")
    ap.add_argument("--fx", type=Path, default=None, help="Optional FX CSV (currency,to_base)")
    ap.add_argument("--base", type=str, default="USD", help="Base currency label for report header")
    ap.add_argument("--out_txt", type=Path, default=Path("report.txt"), help="Output plaintext report")
    ap.add_argument("--out_xlsx", type=Path, default=Path("report_tables.xlsx"), help="Output Excel workbook")
    args = ap.parse_args()

    # Load data and FX
    fx_map = load_fx_map(args.fx)
    df = load_positions(args.positions)

    # Enrich with derived fields
    d = enrich_positions(df, fx_map)

    # KPIs
    kpis = portfolio_kpis(d)

    # Group summaries
    g_ccy = group_summary(d, "currency")
    g_ctry = group_summary(d, "country")
    g_sect = group_summary(d, "sector")

    # Beta bucket distribution
    beta_bins = beta_buckets(d)

    # Concentration metrics (HHI) and Liquidity-adjusted gross
    conc = concentration_metrics(d)
    lag = liquidity_adjusted_gross(d, tau=Thresholds().liq_adjust_tau)

    # Flags and QA
    thr = Thresholds()
    flags = flags_and_watchlist(d, kpis, g_ccy, g_ctry, g_sect, thr)
    qa = qa_checks(d)

    # Write outputs
    used_real_fx = any(val != 1.0 for val in d["fx_to_base"].unique())
    write_text_report(
        out_path=args.out_txt,
        base_ccy=args.base,
        used_real_fx=used_real_fx,
        d=d, kpis=kpis,
        g_ccy=g_ccy, g_ctry=g_ctry, g_sect=g_sect,
        beta_bins=beta_bins,
        lag=lag, hhi=conc["hhi"],
        flags=flags, qa=qa
    )
    write_excel_report(args.out_xlsx, d, g_ccy, g_ctry, g_sect, beta_bins)

    print(f"Wrote {args.out_txt} and {args.out_xlsx}")


if __name__ == "__main__":
    main()


In [ ]:
currency,to_base
EUR,1.08
GBP,1.30
JPY,0.0063
...


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
All-in-one daily report (compact version)
-----------------------------------------
Reads positions.csv and outputs:
  - report.txt (plaintext summary)
  - report_tables.xlsx (drill-downs)

Assumes:
- CSV has columns (case-insensitive, flexible names):
  stock_id, name, ticker, country, currency, sector, industry, sub_industry,
  beta, avg_daily_volume, side, posn_shares, cost_basis_local, market_price_local
- ADV is in shares/day. If in notional, change %ADV formula accordingly.
- FX to base: populate FX_TO_BASE below (or leave blank => 1.0 everywhere).
"""

import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

# ---------------- USER KNOBS ---------------- #
INPUT_CSV   = "positions.csv"
OUT_TXT     = "report.txt"
OUT_XLSX    = "report_tables.xlsx"
BASE_CCY    = "USD"  # label only

# Optional FX map (local -> base). Leave {} to use 1.0 everywhere.
FX_TO_BASE = {
    # "EUR": 1.08, "GBP": 1.30, "JPY": 0.0063, "AUD": 0.66, "CHF": 1.12,
    # "CAD": 0.73, "CNY": 0.14, "HKD": 0.13, "BRL": 0.18, ...
}

# Thresholds for flags
THR = dict(
    beta_drift=0.015,     # 1.5% of gross
    ccy_skew=0.03,        # currency |Net|/Gross > 3%
    bucket_skew=0.05,     # country/sector |Net|/Gross > 5%
    single_name=0.03,     # name |MV|/Gross > 3%
    pctadv=0.10,          # %ADV > 10%
    liq_tau=0.10          # liquidity-adjusted gross threshold (10%)
)

# Beta bucket edges for distribution view
BETA_BUCKETS = [-9, -0.5, 0, 0.5, 9]
# ------------------------------------------- #


# ---------- Helpers ---------- #
def std_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Normalize columns: lower snake_case."""
    out = df.copy()
    out.columns = (out.columns.str.strip().str.lower()
                   .str.replace(r"[\s\-]+", "_", regex=True))
    return out

def load_positions(path: str) -> pd.DataFrame:
    """Read CSV, coerce core columns, and rename to canonical names."""
    df = pd.read_csv(path)
    df = std_cols(df)
    df = df.rename(columns={
        "avg_daily_volume": "adv",
        "posn_shares": "shares",
        "cost_basis_local": "cost",
        "market_price_local": "price",
    })
    # Numeric coercion (ignore if missing)
    for c in ["beta", "adv", "shares", "cost", "price"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    # Ensure grouping cols exist
    for c in ["currency", "country", "sector", "ticker", "name", "industry", "sub_industry"]:
        if c not in df.columns:
            df[c] = "Unknown"
    if "side" not in df.columns:
        df["side"] = np.where(df.get("shares", 0) >= 0, "LONG", "SHORT")
    return df

def enrich(df: pd.DataFrame) -> pd.DataFrame:
    """Compute core derived fields used throughout the report."""
    d = df.copy()
    d["fx_to_base"] = d["currency"].map(FX_TO_BASE).fillna(1.0)
    d["notional_local"] = d["shares"] * d["price"]
    d["mv_base"] = d["notional_local"] * d["fx_to_base"]
    d["abs_mv_base"] = d["mv_base"].abs()
    d["beta_dollars"] = d["beta"] * d["mv_base"]
    d["pct_adv"] = np.where(d["adv"] > 0, np.abs(d["shares"]) / d["adv"], np.nan)
    # P&L (optional)
    if "cost" in d.columns and d["cost"].notna().any():
        d["unrl_pnl"] = (d["price"] - d["cost"]) * d["shares"]
        d["unrl_pnl_pct"] = np.where(d["cost"] != 0, d["price"]/d["cost"] - 1.0, np.nan)
    else:
        d["unrl_pnl"] = np.nan
        d["unrl_pnl_pct"] = np.nan
    return d

def kpis(d: pd.DataFrame) -> dict:
    g = d["abs_mv_base"].sum()
    n = d["mv_base"].sum()
    long_g = d.loc[d["mv_base"] > 0, "mv_base"].sum()
    short_g = d.loc[d["mv_base"] < 0, "mv_base"].abs().sum()
    net_beta = d["beta_dollars"].sum()
    return dict(
        gross=g,
        net=n,
        long_pct=(long_g/g) if g else 0.0,
        short_pct=(short_g/g) if g else 0.0,
        net_beta=net_beta,
        beta_drift=(abs(net_beta)/g) if g else 0.0,
        liq_gross_10pct=(d.loc[d["pct_adv"] > THR["pctadv"], "abs_mv_base"].sum()/(g or 1.0)),
        top10_weight=(d["abs_mv_base"].nlargest(10).sum()/(g or 1.0))
    )

def group_summary(d: pd.DataFrame, by: str) -> pd.DataFrame:
    """Net, Gross, %Gross, Beta$, #names for a dimension."""
    total_gross = d["abs_mv_base"].sum() or 1.0
    g = (d.groupby(by, dropna=False)
          .agg(net=("mv_base", "sum"),
               gross=("abs_mv_base", "sum"),
               beta_dollars=("beta_dollars", "sum"),
               names=("ticker", "nunique"))
          .reset_index())
    g["pct_gross"] = g["gross"] / total_gross
    return g.sort_values("pct_gross", ascending=False)

def beta_bucket_dist(d: pd.DataFrame, edges=BETA_BUCKETS) -> pd.DataFrame:
    labels = [f"[{edges[i]},{edges[i+1]})" for i in range(len(edges)-1)]
    b = d.copy()
    b["beta_bin"] = pd.cut(b["beta"], bins=edges, labels=labels, include_lowest=True)
    g = b.groupby("beta_bin", dropna=False).agg(
        gross=("abs_mv_base", "sum"),
        names=("ticker", "nunique")
    ).reset_index()
    total_gross = d["abs_mv_base"].sum() or 1.0
    g["pct_gross"] = g["gross"] / total_gross
    return g.sort_values("pct_gross", ascending=False)

def concentration_hhi(d: pd.DataFrame) -> float:
    g = d["abs_mv_base"].sum() or 1.0
    w = d["abs_mv_base"] / g
    return float((w**2).sum())

def liquidity_adjusted_gross(d: pd.DataFrame, tau: float) -> float:
    """Down-weight exposures by min(1, %ADV/tau)."""
    scale = np.minimum(1.0, (d["pct_adv"]/tau).fillna(0.0))
    return float((d["abs_mv_base"] * scale).sum())

def make_flags(d: pd.DataFrame, k: dict,
               g_ccy: pd.DataFrame, g_ctry: pd.DataFrame, g_sect: pd.DataFrame) -> list:
    flags = []
    gross = k["gross"] or 1.0

    # Beta drift
    if k["beta_drift"] > THR["beta_drift"]:
        flags.append(f"Market beta drift {k['beta_drift']:.2%} > {THR['beta_drift']:.2%}")

    # Group skews
    for label, g, lim in [("Currency", g_ccy, THR["ccy_skew"]),
                          ("Country", g_ctry, THR["bucket_skew"]),
                          ("Sector",  g_sect, THR["bucket_skew"])]:
        sk = g.loc[g["net"].abs() > lim * gross]
        for _, r in sk.iterrows():
            flags.append(f"{label} skew {r[label.lower()]}: net {r['net']:.0f} ({r['net']/gross:.2%} of gross)")

    # Single-name concentration
    big = d.assign(w=d["abs_mv_base"]/(gross or 1.0)).query("w > @THR['single_name']")
    for _, r in big.iterrows():
        flags.append(f"Single-name concentration {r.get('ticker','<NA>')}: {r['w']:.2%} of gross")

    # Liquidity
    liq = d.loc[d["pct_adv"] > THR["pctadv"]]
    for _, r in liq.iterrows():
        flags.append(f"Liquidity {r.get('ticker','<NA>')}: {r['pct_adv']:.1%} of ADV")

    return flags

def qa_notes(d: pd.DataFrame) -> list:
    notes = []
    for col in ["beta", "adv", "price", "shares", "currency"]:
        if d[col].isna().any():
            notes.append(f"{int(d[col].isna().sum())} rows missing '{col}'")
    bad_adv = (d["adv"] <= 0) | d["adv"].isna()
    if bad_adv.any():
        notes.append(f"{int(bad_adv.sum())} rows have non-positive/missing ADV (pct_adv NaN)")
    extreme_beta = (d["beta"].abs() > 3).sum()
    if extreme_beta:
        notes.append(f"{int(extreme_beta)} rows have |beta|>3 (review)")
    return notes

def fmt_money(x: float) -> str:
    try: return f"{x:,.0f}"
    except: return str(x)


# ---------- Main ---------- #
def main():
    df = load_positions(INPUT_CSV)
    d  = enrich(df)

    # KPIs and analytics
    K = kpis(d)
    g_ccy  = group_summary(d, "currency")
    g_ctry = group_summary(d, "country")
    g_sect = group_summary(d, "sector")
    beta_bins = beta_bucket_dist(d)
    hhi = concentration_hhi(d)
    lag = liquidity_adjusted_gross(d, tau=THR["liq_tau"])
    flags = make_flags(d, K, g_ccy, g_ctry, g_sect)
    qa = qa_notes(d)

    # ---- Write plaintext report ----
    lines = []
    fx_note = "real FX applied" if any(d["fx_to_base"].unique() != 1.0) else "FX=1.0 placeholder"
    lines.append(f"FACTOR-NEUTRAL EQUITIES REPORT | {datetime.now():%Y-%m-%d %H:%M} | Base: {BASE_CCY} | {fx_note}")
    lines.append(f"Rows: {len(d)}")
    lines.append("")
    lines.append("Portfolio KPIs")
    lines.append(f"  Gross: {fmt_money(K['gross'])} | Net: {fmt_money(K['net'])} | "
                 f"Long%: {K['long_pct']:.1%} | Short%: {K['short_pct']:.1%}")
    lines.append(f"  Net beta-$: {fmt_money(K['net_beta'])} | Beta drift: {K['beta_drift']:.2%} of gross")
    lines.append(f"  Liquidity (>%ADV 10%): {K['liq_gross_10pct']:.1%} of gross | "
                 f"Top-10 names: {K['top10_weight']:.1%} of gross")
    lines.append(f"  Liquidity-adjusted gross (LAG): {fmt_money(lag)} | HHI: {hhi:.4f}")
    lines.append("")

    def add_block(title, g, show=10):
        t = g.copy().head(show)
        t["net"] = t["net"].map(fmt_money)
        t["gross"] = t["gross"].map(fmt_money)
        t["beta_dollars"] = t["beta_dollars"].map(fmt_money)
        t["pct_gross"] = (t["pct_gross"]*100).map(lambda v: f"{v:.1f}%")
        lines.append(title)
        lines.append(t.to_string(index=False))
        lines.append("")

    add_block("Top Currency Buckets (by % gross)", g_ccy)
    add_block("Top Country Buckets (by % gross)",  g_ctry)
    add_block("Top Sector Buckets (by % gross)",   g_sect)

    # Beta buckets
    tb = beta_bins.copy()
    tb["gross"] = tb["gross"].map(fmt_money)
    tb["pct_gross"] = (tb["pct_gross"]*100).map(lambda v: f"{v:.1f}%")
    lines.append("Beta Buckets (gross distribution)")
    lines.append(tb.to_string(index=False))
    lines.append("")

    # Optional quick stress/hedge prompts (numbers are already above)
    lines.append("Stress/Hedge Quick Numbers")
    lines.append("  Market stress: ΔPnL ≈ (index move %) × Net Beta-$ / 100")
    lines.append("  Beta hedge notional (if β_index≈1): trade ≈ - Net Beta-$")
    lines.append("  FX stress by currency c: ΔPnL_c ≈ (FX move %) × Net(c) / 100")
    lines.append("")

    # Flags
    lines.append("Watchlist Flags")
    if flags:
        for f in flags: lines.append(f"  - {f}")
    else:
        lines.append("  (None)")
    lines.append("")

    # QA
    if qa:
        lines.append("Data Quality Notes")
        for q in qa: lines.append(f"  - {q}")
        lines.append("")

    Path(OUT_TXT).write_text("\n".join(lines), encoding="utf-8")
    print(f"Wrote {OUT_TXT}")

    # ---- Write Excel workbook ----
    with pd.ExcelWriter(OUT_XLSX, engine="xlsxwriter") as xw:
        g_ccy.to_excel(xw, "by_currency", index=False)
        g_ctry.to_excel(xw, "by_country", index=False)
        g_sect.to_excel(xw, "by_sector", index=False)

        (d.assign(abs_beta=lambda x: x["beta_dollars"].abs())
           .sort_values("abs_beta", ascending=False)
           .drop(columns=["abs_beta"])
           .head(50)
           ).to_excel(xw, "top_beta_contrib", index=False)

        (d.sort_values("abs_mv_base", ascending=False)
           .head(50)
           ).to_excel(xw, "top_exposures", index=False)

        d.sort_values("pct_adv", ascending=False).to_excel(xw, "liquidity", index=False)
        beta_bins.to_excel(xw, "beta_buckets", index=False)
        d.to_excel(xw, "positions_enriched", index=False)

    print(f"Wrote {OUT_XLSX}")


if __name__ == "__main__":
    main()
